In [1]:
from closest_vs_minimal import *

In [2]:
jsonfilename10F = "functioning-false-dim-10-randomperimeter-1000.json"
jsonfilename6F = "functioning-false-dim-6-randomperimeter-1000.json"
jsonfilename4F = "functioning-false-dim-4-randomperimeter-1000.json"
jsonfilename3F = "functioning-false-dim-3-randomperimeter-1000.json"

jsonfilename3T = "functioning-true-dim-3-randomperimeter-1000.json"
jsonfilename4T = "functioning-true-dim-4-randomperimeter-1000.json"
jsonfilename6T = "functioning-true-dim-6-randomperimeter-1000.json"
jsonfilename10T = "functioning-true-dim-10-randomperimeter-1000.json"


In [3]:
for file in [ jsonfilename3T, jsonfilename4T, jsonfilename6T, jsonfilename10T,
    jsonfilename10F, jsonfilename6F,jsonfilename4F, jsonfilename3F]:
    print("\n",file)
    [T,F] = closest_versus_minimal(file)
    print("closest=minimal: True", T, "\tFalse: ", F)


 functioning-true-dim-3-randomperimeter-1000.json
closest=minimal: True 9950 	False:  0

 functioning-true-dim-4-randomperimeter-1000.json
closest=minimal: True 977 	False:  0

 functioning-true-dim-6-randomperimeter-1000.json
closest=minimal: True 94 	False:  0

 functioning-true-dim-10-randomperimeter-1000.json
closest=minimal: True 16 	False:  0

 functioning-false-dim-10-randomperimeter-1000.json
[0, 0, 0, -1, 0, 1, 0, 0, 0, 1] [-1, 0, 1, 0, 0, 1, 0, 0, 1, 1] [-1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, -1.0, 1.0, -1.0] [-0.6289491653442383, -0.5852422714233398, 0.6960563659667969, -0.3621063232421875, -0.2745695114135742, 1.3708477020263672, 0.2796473503112793, 0.615138053894043, 0.3565793037414551, 1.0]
1064.919921875 1260.6171875
[1, -1, 1, -1, 2, -1, -1, 0, 0, 0] [0, 0, 1, 0, 1, 0, 0, 0, 0, 0] [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.6388626098632812, -0.2297685146331787, 1.0, -0.4533805847167969, 1.0022087097167969, -0.3620724678039551, -0.49375200271606445, 0.0629318952